In [40]:
import requests
import mysql.connector
from mysql.connector import Error
import pandas as pd
import mysql_creds

In [7]:
mysql_creds.host

'localhost'

In [8]:
# Set up the API call variables
sample_game_data = []
year = '2021'
season_type = '02' 
max_game_ID = 1290

In [9]:
# Loop over the counter and format the API call
for i in range(1,101):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    sample_game_data.append(data)

## Drop existing tables

In [4]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_drop_Query = """DROP TABLE IF EXISTS team ;\
    DROP TABLE IF EXISTS division;\
    DROP TABLE IF EXISTS conference;
    DROP TABLE IF EXISTS game;"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_drop_Query)
    print("existing tables dropped successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

existing tables dropped successfully 


### Conference and Division data

#### Create the conference table and add all of the tables

In [5]:
conf_info = requests.get(url='https://statsapi.web.nhl.com/api/v1/conferences')
conf_info = conf_info.json()
conf_info = conf_info['conferences']

In [6]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE conference(Id int NOT NULL,\
    name varchar(20) NOT NULL,\
    abbreviation varchar(5) NOT NULL,\
    nameShort varchar(10) NOT NULL,\
    PRIMARY KEY (Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("conference table created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

conference table created successfully 
MySQL connection is closed


In [7]:
def create_conference_insert_statement(conf_stats):
    
    conf_id = conf_stats['id']
    conf_fullname = conf_stats['name']
    conf_abbr = conf_stats['abbreviation']
    conf_nameShort = conf_stats['shortName']
    
    insert_statement = """INSERT INTO conference(Id, name, abbreviation, nameShort)\
    VALUES ({}, '{}', '{}',  '{}')"""\
    .format(conf_id, conf_fullname, conf_abbr, conf_nameShort)
    
    return insert_statement
    
for conference in conf_info:
        
    try:
        connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
        
        mySql_insert_query = create_conference_insert_statement(conference)

        cursor = connection.cursor()
        cursor.execute(mySql_insert_query)
        connection.commit()
        print("{}, inserted successfully into conference table".format(conference['name']))
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into Laptop table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()
#             print("MySQL connection is closed")



Eastern, inserted successfully into conference table
Western, inserted successfully into conference table


#### Division

In [8]:
div_info = requests.get(url='https://statsapi.web.nhl.com/api/v1/divisions')
div_info = div_info.json()
div_info = div_info['divisions']

In [9]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE division(Id int NOT NULL,\
    name varchar(20) NOT NULL,\
    nameShort varchar(5) NOT NULL,\
    abbreviation varchar(5) NOT NULL,\
    conference int NOT NULL,\
    PRIMARY KEY (Id),\
    FOREIGN KEY (conference) REFERENCES conference(Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("division Table created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

division Table created successfully 
MySQL connection is closed


In [10]:
def create_division_insert_statement(div_info):
    
    div_id = div_info['id']
    div_fullname = div_info['name']
    div_abbr = div_info['abbreviation']
    div_nameShort = div_info['nameShort']
    div_conf = div_info['conference']['id']
    
    insert_statement = """INSERT INTO division(Id, name, abbreviation, nameShort, conference)\
    VALUES ({}, '{}', '{}',  '{}', {})"""\
    .format(div_id, div_fullname, div_abbr, div_nameShort, div_conf)
    
    return insert_statement
    
for division in div_info:
        
    try:
        connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
        
        mySql_insert_query = create_division_insert_statement(division)

        cursor = connection.cursor()
        cursor.execute(mySql_insert_query)
        connection.commit()
        print("{}, inserted successfully into division table".format(division['name']))
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into Laptop table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()
#             print("MySQL connection is closed")



Atlantic, inserted successfully into division table
Central, inserted successfully into division table
Metropolitan, inserted successfully into division table
Pacific, inserted successfully into division table


## First I'm going to get information related to the teams

In [11]:
team_info =  requests.get(url='https://statsapi.web.nhl.com/api/v1/teams')
team_info = team_info.json()

### First, build the team table

In [43]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE team(Id int NOT NULL,\
    fullName varchar(50) NOT NULL,\
    abbr varchar(5) NOT NULL,\
    teamName varchar(25) NOT NULL,\
    locationName varchar(25) NOT NULL,\
    division int NOT NULL,\
    conference int NOT NULL,\
    PRIMARY KEY (Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("Team Table created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Team Table created successfully 
MySQL connection is closed


In [44]:
def create_team_insert_statement(team_stats):
    
    team_id = team_stats['id']
    team_fullname = team_stats['name']
    team_abbr = team_stats['abbreviation']
    team_teamName = team_stats['teamName']
    team_locationName = team_stats['locationName']
    team_division = team_stats['division']['id']
    team_conference = team_stats['conference']['id']
    
    insert_statement = """INSERT INTO team(Id, fullName, abbr, teamName, locationName, division, conference)\
    VALUES ({}, '{}', '{}', '{}', '{}', {}, {})"""\
    .format(team_id, team_fullname, team_abbr, team_teamName, team_locationName, team_division, team_conference)
    
    return insert_statement
    
for team in team_info['teams']:
        
    try:
        connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)

        mySql_insert_query = create_team_insert_statement(team)

        cursor = connection.cursor()
        cursor.execute(mySql_insert_query)
        connection.commit()
        print("{}, inserted successfully into team table".format(team['name']))
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into Laptop table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()
#             print("MySQL connection is closed")



New Jersey Devils, inserted successfully into team table
New York Islanders, inserted successfully into team table
New York Rangers, inserted successfully into team table
Philadelphia Flyers, inserted successfully into team table
Pittsburgh Penguins, inserted successfully into team table
Boston Bruins, inserted successfully into team table
Buffalo Sabres, inserted successfully into team table
Montréal Canadiens, inserted successfully into team table
Ottawa Senators, inserted successfully into team table
Toronto Maple Leafs, inserted successfully into team table
Carolina Hurricanes, inserted successfully into team table
Florida Panthers, inserted successfully into team table
Tampa Bay Lightning, inserted successfully into team table
Washington Capitals, inserted successfully into team table
Chicago Blackhawks, inserted successfully into team table
Detroit Red Wings, inserted successfully into team table
Nashville Predators, inserted successfully into team table
St. Louis Blues, inserted

## Player Table

In [137]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE player(ID int NOT NULL,\
    fullName varchar(50) NOT NULL,\
    firstName varchar(25) NOT NULL,\
    lastName varchar(25) NOT NULL,\
    primaryNumner int ,\
    birthDate varchar(15) NOT NULL,\
    birthCity varchar(45) NOT NULL,\
    birthState varchar(25),\
    birthCountry varchar(25) NOT NULL,\
    height int NOT NULL,\
    weight int NOT NULL,\
    active varchar(5) NOT NULL,\
    alternateCaptain varchar(5) ,\
    captain varchar(5),\
    shootsCatches varchar(5) NOT NULL,\
    position varchar(25) NOT NULL,\
    PRIMARY KEY (Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("Team player created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Team player created successfully 
MySQL connection is closed


In [138]:
def get_numeric_height(height):
    
    height_clean = height.strip("'").strip('"').split("' ")
    
    total_inches = (int(height_clean[0]) * 12) + int(height_clean[1])
    
    return total_inches

def get_player_info(player_id):
    
    player_id = str(player_id)
    
    ## use the player info to get the json file
    api_url = 'https://statsapi.web.nhl.com/api/v1/people/' + player_id
    player_data = requests.get(url = api_url)
    player_data = player_data.json()
    player_data = player_data['people'][0]
    
    ## get the specific stats we want
    playerId = player_data['id']
    fullName = player_data['fullName']
    firstName =  player_data['firstName']
    lastName =  player_data['lastName']
    birthDate =  player_data['birthDate']
    birthCity =  player_data['birthCity']
    birthCountry =  player_data['birthCountry']
    height =  get_numeric_height(player_data['height'])
    weight =  player_data['weight']
    active =  player_data['active']
    shootsCatches = player_data['shootsCatches']
    position = player_data['primaryPosition']['name']
    
    
    ## the conditional items (older values don't have this):
    
    
    if 'primaryNumber' in player_data:
        primaryNumner= player_data['primaryNumber']
    else:
        primaryNumner = 'NULL'
    
    
    if 'birthStateProvince' in player_data:
        birthState= player_data['birthStateProvince']
    else:
        birthState = 'NULL'

        
    if 'alternateCaptain' in player_data:
        alternateCaptain= player_data['alternateCaptain']
    else:
        alternateCaptain = 'False'
     
    
    if 'captain' in player_data:
        captain= player_data['captain']
    else:
        captain = 'False'
    
    
    insert_statement = """REPLACE INTO player(Id, fullName, firstName, lastName, primaryNumner, birthDate, birthCity,\
    birthState, birthCountry, height, weight, active, alternateCaptain, captain, shootsCatches, position)\
    VALUES ({}, "{}", "{}", "{}", {}, '{}', "{}", '{}', '{}', {}, {}, '{}', '{}', '{}', '{}', '{}');"""\
    .format(playerId, fullName, firstName, lastName, primaryNumner, birthDate, birthCity,
            birthState, birthCountry, height, weight, active, alternateCaptain, captain, shootsCatches, position)
   
    return insert_statement

In [139]:
seasons = ['20052006', '20062007','20072008','20082009','20092010','20102011','20112012','20122013','20132014',
           '20142015','20152016','20162017','20172018','20182019','20192020','20202021','20212022']

def get_players_from_roster(roster_obj):
    
    ## get our roster object
    roster = roster_obj['teams'][0]['roster']['roster']
    
    ## Return a list of player ID's on that roster
    player_ids = []
    for player in roster:
        
        player_id = player['person']['id']
        player_ids.append(player_id)
        
    return player_ids

## for every season, for every team, if the team existed at the time
for season in seasons:
    for team in range(1,33):
        
        
        ## try to get the team roster
        url = 'https://statsapi.web.nhl.com/api/v1/teams/' + str(team) + '?expand=team.roster&season=' + season
        team_roster_obj = requests.get(url = url)
        team_roster_obj = team_roster_obj.json()
        
        ## If this team roster exists
        if 'message' not in team_roster_obj:
            
            ## Get a list of player IDs from the roster object
            players = get_players_from_roster(team_roster_obj)
                        
            for player in players:
            
                try:
                    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)

                    mySql_insert_query = get_player_info(player)

                    cursor = connection.cursor()
                    cursor.execute(mySql_insert_query)
                    connection.commit()
                    cursor.close()

                except mysql.connector.Error as error:
                    print("Failed to insert record into table {}".format(error))

                finally:
                    if connection.is_connected():
                        connection.close()
                        
                                   
            
            
        
    print(season, 'complete')
            
        

20052006 complete
20062007 complete
20072008 complete
20082009 complete
20092010 complete
20102011 complete
20112012 complete
20122013 complete
20132014 complete
20142015 complete
20152016 complete
20162017 complete
20172018 complete
20182019 complete
20192020 complete
20202021 complete
20212022 complete


## Game Table and player_game table

In [62]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE game(Id int NOT NULL,\
    season varchar(10) NOT NULL,\
    type varchar(5) NOT NULL,\
    startDateTime datetime NOT NULL,\
    endDateTime datetime NOT NULL,\
    awayTeam int NOT NULL,\
    homeTeam int NOT NULL,\
    PRIMARY KEY (Id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("Team game created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Team game created successfully 
MySQL connection is closed


In [63]:
try:
    connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)
    
    
    mySql_Create_Table_Query = """CREATE TABLE player_game(player_id int NOT NULL,\
    team_id int NOT NULL,\
    game_id int NOT NULL,\
    PRIMARY KEY (player_id,team_id,game_id))"""
    
    
    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("Team player_game created successfully ")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Team player_game created successfully 
MySQL connection is closed


In [64]:
def create_game_insert_statement(game_info):
    game_id = game_info['gameData']['game']['pk']
    game_season = game_info['gameData']['game']['season']
    game_type = game_info['gameData']['game']['type']
    game_startDateTime = pd.to_datetime(game_info['gameData']['datetime']['dateTime']).strftime('%Y-%m-%d %H:%M:%S')
    game_endDateTime = pd.to_datetime(game_info['gameData']['datetime']['endDateTime']).strftime('%Y-%m-%d %H:%M:%S')
    away_team = game_info['gameData']['teams']['away']['id']
    home_team = game_info['gameData']['teams']['home']['id']
    
    insert_statement = """INSERT INTO game(Id, season, type, startDateTime, endDateTime, awayTeam, homeTeam)\
    VALUES ({}, '{}', '{}', '{}', '{}', {}, {})"""\
    .format(game_id, game_season, game_type, game_startDateTime, game_endDateTime, away_team, home_team)
    
    return insert_statement

def get_player_game_row(game_data):
    
    insert_statements = []
    
    ## get the game_id
    game_id = game_data['gameData']['game']['pk']
    
    ## iterate through all of the players
    for player in game_data['gameData']['players'].keys():
        
        player_dat =  game_data['gameData']['players'][player]
        
        ## Get the player ID and team ID
        player_id = player_dat['id']
        team_id = player_dat['currentTeam']['id']
        
        ## Create the insert statement and add it to the return list
        insert_statement = """INSERT INTO player_game(player_id, team_id, game_id)\
        VALUES ({},  {}, {})"""\
        .format(player_id, team_id, game_id)
        
        insert_statements.append(insert_statement)
    
    return insert_statements

for game in sample_game_data:
    
    ## First, upload the game table to the database
    try:
        connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)

        mySql_insert_query = create_game_insert_statement(game)

        cursor = connection.cursor()
        cursor.execute(mySql_insert_query)
        connection.commit()
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()
            
    ## Then, we upload the player_game rows
    player_game_inserts = get_player_game_row(game)

    for player_game_insert in player_game_inserts:
        
        try:
            connection = mysql.connector.connect(host=mysql_creds.host,
                                         database=mysql_creds.database,
                                         user=mysql_creds.user,
                                         password=mysql_creds.password)

            mySql_insert_query = player_game_insert

            cursor = connection.cursor()
            cursor.execute(mySql_insert_query)
            connection.commit()
            cursor.close()

        except mysql.connector.Error as error:
            print("Failed to insert record into table {}".format(error))

        finally:
            if connection.is_connected():
                connection.close()
                
    print(game['gamePk'])
                


2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020001
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020002
2021020003
2021020003
2021020003
2021020003

KeyError: 'currentTeam'

In [37]:
def get_player_game_row(game_data):
    
    insert_statements = []
    
    ## get the game_id
    game_id = game_data['gameData']['game']['pk']
    
    ## iterate through all of the players
    for player in game_data['gameData']['players'].keys():
        
        player_dat =  game_data['gameData']['players'][player]
        
        ## Get the player ID and team ID
        player_id = player_dat['id']
        team_id = player_dat['currentTeam']['id']
        
        ## Create the insert statement and add it to the return list
        insert_statement = """INSERT INTO player_game(player_id, team_id, game_id)\
        VALUES ({},  {}, {})"""\
        .format(player_id, team_id, game_id)
        
        insert_statements.append(insert_statement)
    
    return insert_statements

In [72]:
sample_game_data[11]['gameData']['players'].keys()#['ID8470604']['currentTeam']

for i in sample_game_data[11]['gameData']['players'].keys():
    
    print(sample_game_data[11]['gameData']['players'][i]['currentTeam'])

{'id': 12, 'name': 'Carolina Hurricanes', 'link': '/api/v1/teams/12', 'triCode': 'CAR'}
{'id': 12, 'name': 'Carolina Hurricanes', 'link': '/api/v1/teams/12', 'triCode': 'CAR'}
{'id': 12, 'name': 'Carolina Hurricanes', 'link': '/api/v1/teams/12', 'triCode': 'CAR'}
{'id': 2, 'name': 'New York Islanders', 'link': '/api/v1/teams/2', 'triCode': 'NYI'}
{'id': 12, 'name': 'Carolina Hurricanes', 'link': '/api/v1/teams/12', 'triCode': 'CAR'}
{'id': 12, 'name': 'Carolina Hurricanes', 'link': '/api/v1/teams/12', 'triCode': 'CAR'}
{'id': 2, 'name': 'New York Islanders', 'link': '/api/v1/teams/2', 'triCode': 'NYI'}
{'id': 12, 'name': 'Carolina Hurricanes', 'link': '/api/v1/teams/12', 'triCode': 'CAR'}
{'id': 2, 'name': 'New York Islanders', 'link': '/api/v1/teams/2', 'triCode': 'NYI'}
{'id': 2, 'name': 'New York Islanders', 'link': '/api/v1/teams/2', 'triCode': 'NYI'}
{'id': 12, 'name': 'Carolina Hurricanes', 'link': '/api/v1/teams/12', 'triCode': 'CAR'}
{'id': 12, 'name': 'Carolina Hurricanes', 'l

KeyError: 'currentTeam'

In [92]:
sample_game_data[11]['gameData']['players'].keys()#['players']['ID8473463']

dict_keys(['ID8482093', 'ID8478904', 'ID8480829', 'ID8476429', 'ID8473533', 'ID8476869', 'ID8478009', 'ID8478427', 'ID8475314', 'ID8475754', 'ID8475799', 'ID8476921', 'ID8475855', 'ID8480865', 'ID8477488', 'ID8474573', 'ID8480789', 'ID8475883', 'ID8473463', 'ID8478451', 'ID8474013', 'ID8474090', 'ID8480222', 'ID8465009', 'ID8476917', 'ID8474613', 'ID8477506', 'ID8476419', 'ID8476958', 'ID8473504', 'ID8477527', 'ID8470610', 'ID8472382', 'ID8478445', 'ID8477950', 'ID8476882', 'ID8476389', 'ID8478463', 'ID8475151', 'ID8479356', 'ID8475231', 'ID8480830', 'ID8480039'])

In [ ]:
?expand=team.roster&season=20142015

In [85]:
url = "https://statsapi.web.nhl.com/api/v1/teams/?expand=team.roster"
team_data = requests.get(url = url)
team_data = team_data.json()
team_data['teams'][1]['roster']['roster']

[{'person': {'id': 8465009,
   'fullName': 'Zdeno Chara',
   'link': '/api/v1/people/8465009'},
  'jerseyNumber': '33',
  'position': {'code': 'D',
   'name': 'Defenseman',
   'type': 'Defenseman',
   'abbreviation': 'D'}},
 {'person': {'id': 8470610,
   'fullName': 'Zach Parise',
   'link': '/api/v1/people/8470610'},
  'jerseyNumber': '11',
  'position': {'code': 'L',
   'name': 'Left Wing',
   'type': 'Forward',
   'abbreviation': 'LW'}},
 {'person': {'id': 8471239,
   'fullName': 'Cory Schneider',
   'link': '/api/v1/people/8471239'},
  'jerseyNumber': '35',
  'position': {'code': 'G',
   'name': 'Goalie',
   'type': 'Goalie',
   'abbreviation': 'G'}},
 {'person': {'id': 8472382,
   'fullName': 'Andy Greene',
   'link': '/api/v1/people/8472382'},
  'jerseyNumber': '4',
  'position': {'code': 'D',
   'name': 'Defenseman',
   'type': 'Defenseman',
   'abbreviation': 'D'}},
 {'person': {'id': 8473504,
   'fullName': 'Cal Clutterbuck',
   'link': '/api/v1/people/8473504'},
  'jerseyNumb

In [75]:
api_url = 'https://statsapi.web.nhl.com/api/v1/people/8473463'
player_data = requests.get(url = api_url)
player_data = player_data.json()

player_data

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2022. All Rights Reserved.',
 'people': [{'id': 8473463,
   'fullName': 'Leo Komarov',
   'link': '/api/v1/people/8473463',
   'firstName': 'Leo',
   'lastName': 'Komarov',
   'primaryNumber': '47',
   'birthDate': '1987-01-23',
   'birthCity': 'Narva',
   'birthCountry': 'EST',
   'nationality': 'FIN',
   'height': '5\' 11"',
   'weight': 205,
   'active': False,
   'rookie': False,
   'shootsCatches': 'L',
   'rosterStatus': 'N',
   'primaryPosition': {'code': 'R',
    'name': 'Right Wing',
    'type': 'Forward',
    'abbreviation': 'RW'}}]}

In [335]:
s = player_data['people'][0]['height']
s.split(' ' )

["6'", '0"']

In [93]:
year

'2021'

### Game data

In [96]:
game_data_old = []
# Loop over the counter and format the API call
for i in range(1,6):
    r = requests.get(url='http://statsapi.web.nhl.com/api/v1/game/'
        + '2012' + season_type +str(i).zfill(4)+'/feed/live')
    data = r.json()
    game_data_old.append(data)

In [102]:
game_data_old[0]['gameData']['players']

{'ID8465951': {'id': 8465951,
  'fullName': 'Matt Cooke',
  'link': '/api/v1/people/8465951',
  'firstName': 'Matt',
  'lastName': 'Cooke',
  'primaryNumber': '24',
  'birthDate': '1978-09-07',
  'birthCity': 'Belleville',
  'birthStateProvince': 'ON',
  'birthCountry': 'CAN',
  'nationality': 'CAN',
  'height': '5\' 11"',
  'weight': 216,
  'active': False,
  'rookie': False,
  'shootsCatches': 'L',
  'rosterStatus': 'N',
  'primaryPosition': {'code': 'L',
   'name': 'Left Wing',
   'type': 'Forward',
   'abbreviation': 'LW'}},
 'ID8468524': {'id': 8468524,
  'fullName': 'Ilya Bryzgalov',
  'link': '/api/v1/people/8468524',
  'firstName': 'Ilya',
  'lastName': 'Bryzgalov',
  'primaryNumber': '80',
  'birthDate': '1980-06-22',
  'birthCity': 'Togliatti',
  'birthCountry': 'RUS',
  'nationality': 'RUS',
  'height': '6\' 3"',
  'weight': 213,
  'active': False,
  'rookie': False,
  'shootsCatches': 'L',
  'rosterStatus': 'N',
  'primaryPosition': {'code': 'G',
   'name': 'Goalie',
   'ty

In [127]:
game_data_old[2]['liveData']['plays']['allPlays'][39]

{'players': [{'player': {'id': 8471208,
    'fullName': 'Dustin Penner',
    'link': '/api/v1/people/8471208'},
   'playerType': 'Shooter'}],
 'result': {'event': 'Missed Shot',
  'eventCode': 'LAK72',
  'eventTypeId': 'MISSED_SHOT',
  'description': 'Dustin Penner - Wide of Net'},
 'about': {'eventIdx': 39,
  'eventId': 72,
  'period': 1,
  'periodType': 'REGULAR',
  'ordinalNum': '1st',
  'periodTime': '07:03',
  'periodTimeRemaining': '12:57',
  'dateTime': '2013-01-19T20:40:03Z',
  'goals': {'away': 1, 'home': 0}},
 'coordinates': {'x': -52.0, 'y': -20.0},
 'team': {'id': 26,
  'name': 'Los Angeles Kings',
  'link': '/api/v1/teams/26',
  'triCode': 'LAK'}}

In [125]:
event_type_dict = dict()

for event in game_data[0]['liveData']['plays']['allPlays']:
        
    if event['result']['event'] not in event_type_dict:
        event_type_dict[event['result']['event']] = 1
        
        
    if event['result']['event'] in event_type_dict:
        event_type_dict[event['result']['event']] += 1


In [126]:
event_type_dict

{'Game Scheduled': 2,
 'Period Ready': 4,
 'Period Start': 4,
 'Faceoff': 70,
 'Hit': 60,
 'Stoppage': 57,
 'Shot': 56,
 'Takeaway': 9,
 'Blocked Shot': 27,
 'Missed Shot': 25,
 'Giveaway': 8,
 'Period End': 4,
 'Period Official': 4,
 'Goal': 9,
 'Penalty': 3,
 'Game End': 2,
 'Game Official': 2}